In [1]:
# Ref:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [2]:
!pip -q install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


In [3]:
import json
import os
import re
from google.colab import userdata

# Define the API clients

In [4]:
from groq import Groq

groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)


# What is the objective?

Create a ReAct pattern using agent functions instead of LangGraph.<br>
The system will be able to respond to multi-stage questions that require the model to call different functions before the final answer can be output.<br>
Questions are about dog weights.



# Create a list of agents

AGENTS
1. chat_agent
2. router_agent
3. research_agent
4. final_answer_agent

TOOLS
- Function that returns an avaerage dog weight given a dog breed
- Calculate function that returns answers to calculations

BLOCKS
- ReAct block

# Helper functions

In [5]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [6]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



In [21]:
def initialize_message_history(system_message):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history



# Set up the LLM

In [7]:
def make_llm_api_call(message_history):

    """
    Makes a call to the Llama3 model on Groq.
    Args:
        message_history (List of dicts): The message history
    Returns:
        response_text: (str): The text response from the LLM
    """

    response = groq_client.chat.completions.create(
                        messages=message_history,
                        model="llama3-70b-8192",
                    )

    response_text = response.choices[0].message.content

    return response_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response = make_llm_api_call(message_history)

print(response)

My name is Molly! Nice to meet you!


# Set up the tools



In [8]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

# Set up the system messages

In [9]:
chat_agent_system_message = """
You are a helpful assistant.

1. You provide polite amswers to simple questions.
If the user's input requires only a simple answer then output your answer as JSON.

Example session:

Question: Hello. How are you?

You output:

{
"Answer": "I'm fine thanks.",
"Status": "DONE"
}



2. You can also run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Output your response as a JSON string.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

You can only call one action at a time.


Example session:

Question: How much does a Bulldog weigh?
{
    "Thought": "I should look the dogs weight using average_dog_weight",
    "Action": {"function":"average_dog_weight", "input": "Bulldog"},
    "Status": "PAUSE"
}

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:
{
    "Answer": "A bulldog weights 51 lbs",
    "Status": "DONE"
}
""".strip()



# Set up the Agents

In [10]:
def run_chat_agent(message_history):

    print("---CHAT AGENT---")

    # Prompt the llm
    response = make_llm_api_call(message_history)

    print(response)

    return response



# Example

user_query = "How much does a toy poodle weigh?"

message_history = create_message_history(chat_agent_system_message, user_query)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Update message history
message = [{"role": "assistant", "content": response}]
message_history.append(message)


---CHAT AGENT---
{
    "Thought": "I should look the dog's weight using average_dog_weight",
    "Action": {"function":"average_dog_weight", "input": "Toy Poodle"},
    "Status": "PAUSE"
}


In [11]:
def run_router_agent(llm_response):

    """
    Route to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTER AGENT---")


    # Extract the status
    json_response = json.loads(llm_response)
    status = json_response['Status']
    #status = extract_json_str_value(response, "Status").strip()

    print("Status:", status)

    if status == 'PAUSE':
        print("Route: to_research_agent")
        return "to_research_agent"

    elif status == 'DONE':
        print("Route: to_final_answer")
        return "to_final_answer"



# Example

user_query = "How much does a toy poodle weigh?"

message_history = create_message_history(chat_agent_system_message, user_query)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Update message history
message = [{"role": "assistant", "content": response}]
message_history.append(message)

# Run router_agent
route = run_router_agent(response)


---CHAT AGENT---
{
    "Thought": "I should look the dog's weight using average_dog_weight",
    "Action": {"function":"average_dog_weight", "input": "Toy Poodle"},
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent


In [12]:
def run_research_agent(llm_response):

    print("---RESEARCH AGENT---")

    # Extract the status
    json_response = json.loads(llm_response)
    action_dict = json_response['Action']
    func_to_run = action_dict['function']
    func_input = action_dict['input']

    if func_to_run == "average_dog_weight":
        answer = average_dog_weight(func_input)
    else:
        answer = calculate(func_input)

    print("func_to_run:", func_to_run)
    print("func_arg:", func_input)
    print("Output:", answer)

    return answer



# Example

user_query = "How much does a toy poodle weigh?"

message_history = create_message_history(chat_agent_system_message, user_query)

# Prompt the chat_agent
llm_response = run_chat_agent(message_history)

# Update message history
message = {"role": "assistant", "content": llm_response}
message_history.append(message)

# Run router_agent
route = run_router_agent(llm_response)


if route == "to_research_agent":
    answer = run_research_agent(llm_response)

    # Update message history
    message = {"role": "user", "content": f"Observation: {answer}"}
    message_history.append(message)



---CHAT AGENT---
{
    "Thought": "I should look up the average weight of a toy poodle using average_dog_weight",
    "Action": {"function":"average_dog_weight", "input": "Toy Poodle"},
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: average_dog_weight
func_arg: Toy Poodle
Output: a toy poodles average weight is 7 lbs


In [13]:
def run_final_answer_agent(llm_response):

    print("---FINAL ANSWER AGENT---")

    json_response = json.loads(llm_response)
    final_answer = json_response['Answer']

    print("Final answer:", final_answer)

# Run the system

In [17]:
user_query1 = "How much does a Scottish Terrier weigh?"

user_query2 = "How much does a Scottish Terrier and a Border Collie weigh?"

# For this question the model can try to call all
# the functions it needs at the same time.
user_query3 = "What is the average weight of a Scottish Terrier and a Border Collie?"


message_history = create_message_history(chat_agent_system_message, user_query1)


for i in range(0,5):

    # Prompt the chat_agent
    llm_response = run_chat_agent(message_history)

    # Update message history
    message = {"role": "assistant", "content": llm_response}
    message_history.append(message)

    # Run router_agent
    route = run_router_agent(llm_response)


    if route == "to_research_agent":

        answer = run_research_agent(llm_response)

        # Update message history
        message = {"role": "user", "content": f"Observation: {answer}"}
        message_history.append(message)

    else:

        run_final_answer_agent(llm_response)

        break



---CHAT AGENT---
{
    "Thought": "I should look up the dog's weight using average_dog_weight",
    "Action": {"function":"average_dog_weight", "input": "Scottish Terrier"},
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: average_dog_weight
func_arg: Scottish Terrier
Output: Scottish Terriers average 20 lbs
---CHAT AGENT---
{
    "Answer": "A Scottish Terrier weighs approximately 20 lbs",
    "Status": "DONE"
}
---ROUTER AGENT---
Status: DONE
Route: to_final_answer
---FINAL ANSWER AGENT---
Final answer: A Scottish Terrier weighs approximately 20 lbs


In [18]:
# Prompting the user for input
user_input = input("Please enter something: ")

Please enter something: Hello


In [19]:
user_input

'Hello'

# Run infinitely with a user input field

In [27]:
# Prompting the user for input
#user_input = input("Please enter something: ")

message_history = initialize_message_history(chat_agent_system_message)

while True:

    print()
    print("==========")
    user_input = input("Enter something (or 'q' to quit): ")
    print("==========")

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop

    # Update message history
    message = {"role": "user", "content": user_input}
    message_history.append(message)


    for i in range(0,5):

        # Prompt the chat_agent
        llm_response = run_chat_agent(message_history)

        # Update message history
        message = {"role": "assistant", "content": llm_response}
        message_history.append(message)

        # Run router_agent
        route = run_router_agent(llm_response)


        if route == "to_research_agent":

            answer = run_research_agent(llm_response)

            # Update message history
            message = {"role": "user", "content": f"Observation: {answer}"}
            message_history.append(message)

        else:

            run_final_answer_agent(llm_response)

            break




Enter something (or 'q' to quit): Hi. Do you know about dogs?
---CHAT AGENT---
{
"Answer": "Yes, I know some things about dogs. I can provide information on dog breeds, their characteristics, and some interesting facts.",
"Status": "DONE"
}
---ROUTER AGENT---
Status: DONE
Route: to_final_answer
---FINAL ANSWER AGENT---
Final answer: Yes, I know some things about dogs. I can provide information on dog breeds, their characteristics, and some interesting facts.

Enter something (or 'q' to quit): How much does a borrder collie weigh?
---CHAT AGENT---
{
    "Thought": "I should look up the average weight of a Border Collie using average_dog_weight",
    "Action": {"function":"average_dog_weight", "input": "Border Collie"},
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: average_dog_weight
func_arg: Border Collie
Output: a Border Collies average weight is 37 lbs
---CHAT AGENT---
{
    "Answer": "A Border Collie's average we